# Recovery/Planning Notebook
**ST-GNN + Uncertainty from Weather Ensembles (GenCast / FourCastNet3 style)**

Template for:
- Load facility + vulnerability CSVs (Shelters, Hospitals, Schools, Vulnerability grid)
- Build a distance-based graph
- Load weather ensemble time series (GenCast/FourCastNet3 outputs interpolated to your nodes)
- Train an ST-GNN model with quantile outputs (P10/P50/P90)
- Predict recovery distributions and run a simple planning allocation


In [1]:
# 0) Install dependencies (run once)
!pip -q install pandas numpy scikit-learn torch torch-geometric geopy


In [2]:
# 1) Imports
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from sklearn.neighbors import BallTree
from torch_geometric.nn import GCNConv


## 2) File paths (edit these)

In [3]:
SHELTERS_CSV = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\data\\raw\\facilities\\shelters.csv"
HOSPITALS_CSV = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\data\\raw\\facilities\\hospitals.csv"
SCHOOLS_CSV   = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\data\\raw\\facilities\\schools.csv"
VULN_GRID_CSV = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\data\\raw\\vulnerability\\\\vulnerability_grid_clean.csv"

# Optional (for training)
WEATHER_ENSEMBLE_CSV = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\data\\raw\\weather_ensemble.csv"  # t, node_id, ens, wind10m, precip, mslp, gust, ...
RECOVERY_LABELS_CSV  = "C:\\Users\\Adrija\\Downloads\\DFGCN\\data\\raw\\recovery_labels.csv"   # t, node_id, recovery_index

## 3) Load CSVs and build node table

In [4]:
def load_facilities(shelters_csv, hospitals_csv, schools_csv, vuln_grid_csv):
    shelters = pd.read_csv(shelters_csv)
    hospitals = pd.read_csv(hospitals_csv)
    schools = pd.read_csv(schools_csv)
    grid = pd.read_csv(vuln_grid_csv)

    # Standardize facilities
    for df, kind in [(shelters, "shelter"), (hospitals, "hospital"), (schools, "school")]:
        df["kind"] = kind
        if "id" not in df.columns:
            df["id"] = [f"{kind}_{i}" for i in range(len(df))]

    # Standardize grid
    if "cell_id" not in grid.columns:
        grid["cell_id"] = [f"cell_{i}" for i in range(len(grid))]
    grid["kind"] = "grid"
    grid = grid.rename(columns={"cell_id": "id"})

    nodes = pd.concat([grid, shelters, hospitals, schools], ignore_index=True)

    if "lat" not in nodes.columns or "lon" not in nodes.columns:
        raise ValueError("All inputs must contain 'lat' and 'lon' columns.")

    nodes["lat"] = pd.to_numeric(nodes["lat"], errors="coerce")
    nodes["lon"] = pd.to_numeric(nodes["lon"], errors="coerce")
    nodes = nodes.dropna(subset=["lat","lon"]).reset_index(drop=True)

    return nodes

nodes = load_facilities(SHELTERS_CSV, HOSPITALS_CSV, SCHOOLS_CSV, VULN_GRID_CSV)
print("Nodes:", nodes.shape)
nodes.head()


ValueError: All inputs must contain 'lat' and 'lon' columns.

## 4) Build graph edges (distance-based)

In [ ]:
def haversine_graph(nodes_df, R_km=15.0, max_neighbors=20):
    coords = np.deg2rad(nodes_df[["lat","lon"]].values)
    tree = BallTree(coords, metric="haversine")
    R = R_km / 6371.0

    ind, dist = tree.query_radius(coords, r=R, return_distance=True, sort_results=True)

    edge_src, edge_dst, edge_w = [], [], []
    for i, (nbrs, dists) in enumerate(zip(ind, dist)):
        nbrs = nbrs[:max_neighbors]
        dists = dists[:max_neighbors]
        for j, d in zip(nbrs, dists):
            if i == j:
                continue
            edge_src.append(i)
            edge_dst.append(j)
            edge_w.append(1.0 / (d + 1e-6))

    edge_index = np.vstack([edge_src, edge_dst]).astype(np.int64)
    edge_weight = np.array(edge_w, dtype=np.float32)
    return edge_index, edge_weight

edge_index, edge_weight = haversine_graph(nodes, R_km=15, max_neighbors=20)
print("edge_index:", edge_index.shape, "edge_weight:", edge_weight.shape)


## 5) Static features (per node)

In [ ]:
def build_static_features(nodes_df):
    numeric_cols = nodes_df.select_dtypes(include=[np.number]).columns.tolist()
    feat_cols = [c for c in numeric_cols if c not in ["lat","lon"]]
    Xs = nodes_df[feat_cols].fillna(0.0).to_numpy(dtype=np.float32)
    return Xs, feat_cols

X_static, static_cols = build_static_features(nodes)
print("X_static:", X_static.shape)
print("Example static columns:", static_cols[:10])


## 6) Load weather ensembles to tensor (T, N, F, E)

CSV must contain: `t`, `node_id`, `ens` and dynamic columns.


In [ ]:
def load_weather_ensembles(weather_csv, nodes_df, dyn_cols=("wind10m","precip","mslp","gust")):
    w = pd.read_csv(weather_csv)
    node_to_idx = {nid:i for i, nid in enumerate(nodes_df["id"].tolist())}

    T = int(w["t"].max()) + 1
    E = int(w["ens"].max()) + 1
    N = len(nodes_df)
    F = len(dyn_cols)

    Xd = np.zeros((T, N, F, E), dtype=np.float32)

    for _, row in w.iterrows():
        t = int(row["t"]); e = int(row["ens"])
        nid = row["node_id"]
        if nid not in node_to_idx:
            continue
        n = node_to_idx[nid]
        Xd[t, n, :, e] = np.array([row[c] for c in dyn_cols], dtype=np.float32)

    return Xd

# Uncomment when you have the file:
# X_dynamic = load_weather_ensembles(WEATHER_ENSEMBLE_CSV, nodes)
# print("X_dynamic:", X_dynamic.shape)


## 7) Load recovery labels (T, N)

In [ ]:
def load_recovery_labels(labels_csv, nodes_df, y_col="recovery_index"):
    ydf = pd.read_csv(labels_csv)
    node_to_idx = {nid:i for i, nid in enumerate(nodes_df["id"].tolist())}

    T = int(ydf["t"].max()) + 1
    N = len(nodes_df)
    Y = np.full((T, N), np.nan, dtype=np.float32)

    for _, row in ydf.iterrows():
        t = int(row["t"])
        nid = row["node_id"]
        if nid not in node_to_idx:
            continue
        n = node_to_idx[nid]
        Y[t, n] = float(row[y_col])

    Y = pd.DataFrame(Y).fillna(method="ffill").fillna(0.0).to_numpy(dtype=np.float32)
    return Y

# Uncomment when you have the file:
# Y = load_recovery_labels(RECOVERY_LABELS_CSV, nodes, y_col="recovery_index")
# print("Y:", Y.shape)


## 8) ST-GNN model (GCN + GRU) with quantile outputs

In [ ]:
class STGNNQuantile(nn.Module):
    def __init__(self, static_dim, dyn_dim, hidden=64, quantiles=(0.1, 0.5, 0.9)):
        super().__init__()
        self.quantiles = quantiles
        self.gcn1 = GCNConv(static_dim + dyn_dim, hidden)
        self.gcn2 = GCNConv(hidden, hidden)
        self.gru = nn.GRU(input_size=hidden, hidden_size=hidden, batch_first=True)
        self.head = nn.Linear(hidden, len(quantiles))

    def forward(self, X_static, X_dyn, edge_index, edge_weight=None):
        # X_static: (N,S), X_dyn: (T,N,F) for one ensemble
        T, N, F = X_dyn.shape
        h_seq = []
        for t in range(T):
            x = torch.cat([X_static, X_dyn[t]], dim=-1)
            h = torch.relu(self.gcn1(x, edge_index, edge_weight))
            h = torch.relu(self.gcn2(h, edge_index, edge_weight))
            h_seq.append(h)
        H = torch.stack(h_seq, dim=1)  # (N,T,H)
        out, _ = self.gru(H)
        q = self.head(out)             # (N,T,Q)
        return q

def quantile_loss(pred_q, y, quantiles=(0.1,0.5,0.9)):
    losses = []
    for qi, qv in enumerate(quantiles):
        e = y - pred_q[..., qi]
        losses.append(torch.max((qv - 1) * e, qv * e).mean())
    return sum(losses)


## 9) Train across ensembles (uncertainty propagation)

In [ ]:
def train(model, optimizer, X_static, X_dynamic, Y, edge_index, edge_weight=None, epochs=20):
    device = next(model.parameters()).device
    Xs = torch.tensor(X_static, dtype=torch.float32, device=device)
    edge_index_t = torch.tensor(edge_index, dtype=torch.long, device=device)
    ew = torch.tensor(edge_weight, dtype=torch.float32, device=device) if edge_weight is not None else None

    Yt = torch.tensor(Y, dtype=torch.float32, device=device).transpose(0, 1)  # (N,T)
    T, N, F, E = X_dynamic.shape

    for ep in range(epochs):
        model.train()
        optimizer.zero_grad()

        total = 0.0
        for e in range(E):
            Xd = torch.tensor(X_dynamic[..., e], dtype=torch.float32, device=device)  # (T,N,F)
            q = model(Xs, Xd, edge_index_t, ew)  # (N,T,Q)
            total += quantile_loss(q, Yt, model.quantiles)

        loss = total / E
        loss.backward()
        optimizer.step()
        print(f"epoch {ep+1}/{epochs} loss={loss.item():.4f}")

# Example (uncomment after loading X_dynamic and Y):
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = STGNNQuantile(static_dim=X_static.shape[1], dyn_dim=X_dynamic.shape[2], hidden=64).to(device)
# optim = torch.optim.Adam(model.parameters(), lr=1e-3)
# train(model, optim, X_static, X_dynamic, Y, edge_index, edge_weight, epochs=20)


## 10) Inference: mean/std + p10/p90 across ensembles

In [ ]:
@torch.no_grad()
def predict_distribution(model, X_static, X_dynamic, edge_index, edge_weight=None):
    device = next(model.parameters()).device
    Xs = torch.tensor(X_static, dtype=torch.float32, device=device)
    edge_index_t = torch.tensor(edge_index, dtype=torch.long, device=device)
    ew = torch.tensor(edge_weight, dtype=torch.float32, device=device) if edge_weight is not None else None

    T, N, F, E = X_dynamic.shape
    preds = []
    for e in range(E):
        Xd = torch.tensor(X_dynamic[..., e], dtype=torch.float32, device=device)  # (T,N,F)
        q = model(Xs, Xd, edge_index_t, ew)  # (N,T,Q)
        preds.append(q.cpu().numpy())

    preds = np.stack(preds, axis=0)  # (E,N,T,Q)
    q50 = preds[..., 1]              # (E,N,T)

    mean = q50.mean(axis=0)          # (N,T)
    std  = q50.std(axis=0)           # (N,T)
    p10  = np.quantile(q50, 0.10, axis=0)
    p90  = np.quantile(q50, 0.90, axis=0)

    return {"mean": mean, "std": std, "p10": p10, "p90": p90, "raw": preds}

# Example:
# dist = predict_distribution(model, X_static, X_dynamic, edge_index, edge_weight)
# dist["mean"].shape, dist["std"].shape


## 11) Planning baseline: allocate resources by risk

In [ ]:
def plan_resources(nodes_df, recovery_mean, vulnerability_col="SVI", budget=100, horizon_t=0):
    N = len(nodes_df)
    svi = pd.to_numeric(nodes_df.get(vulnerability_col, 0.0), errors="coerce").fillna(0.0).to_numpy()

    next_step = recovery_mean[:, horizon_t]  # (N,)
    risk = svi * (1.0 - next_step)

    risk = np.maximum(risk, 0.0)
    alloc = np.zeros(N, dtype=np.float32) if risk.sum() == 0 else budget * (risk / risk.sum())

    out = nodes_df[["id","kind","lat","lon"]].copy()
    out["risk_score"] = risk
    out["resource_alloc"] = alloc
    return out.sort_values("risk_score", ascending=False)

# Example:
# plan_df = plan_resources(nodes, dist["mean"], vulnerability_col="SVI", budget=100, horizon_t=0)
# plan_df.head(20)
